In [17]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [18]:
# 쓸 기능들 import 합니다.
from ctypes import sizeof
from doctest import set_unittest_reportflags
from tkinter import font
from typing import Counter
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sympy import rotations
import os

In [28]:
# 파일 불러옵니다.
Seoul_22_1Q = pd.read_csv("/content/drive/MyDrive/소상공인시장진흥공단_상가(상권)정보_서울_202203.csv")
Gyeonggi_22_1Q = pd.read_csv("/content/drive/MyDrive/소상공인시장진흥공단_상가(상권)정보_경기_202203.csv")

In [29]:
# 불필요하거나 중복되는 변수는 drop 해줍니다.
Seoul_22_1Q.drop(labels=['상권업종대분류코드' ,'상권업종중분류코드','상권업종소분류코드','표준산업분류코드','시도코드','시도명','시군구코드','행정동코드','행정동명','법정동코드','법정동명','지번코드','대지구분코드','대지구분명','지번본번지','지번부번지','도로명코드','도로명','건물본번지','건물부번지','건물관리번호','건물명','도로명주소','구우편번호','신우편번호'], axis=1, inplace=True)
Seoul_22_1Q.info()

Gyeonggi_22_1Q.drop(labels=['상권업종대분류코드' ,'상권업종중분류코드','상권업종소분류코드','표준산업분류코드','시도코드','시도명','시군구코드','행정동코드','행정동명','법정동코드','법정동명','지번코드','대지구분코드','대지구분명','지번본번지','지번부번지','도로명코드','도로명','건물본번지','건물부번지','건물관리번호','건물명','도로명주소','구우편번호','신우편번호'], axis=1, inplace=True)
Gyeonggi_22_1Q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330827 entries, 0 to 330826
Data columns (total 14 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   상가업소번호    330827 non-null  int64  
 1   상호명       330827 non-null  object 
 2   지점명       58133 non-null   object 
 3   상권업종대분류명  330827 non-null  object 
 4   상권업종중분류명  330827 non-null  object 
 5   상권업종소분류명  330827 non-null  object 
 6   표준산업분류명   310386 non-null  object 
 7   시군구명      330827 non-null  object 
 8   지번주소      330827 non-null  object 
 9   동정보       34269 non-null   object 
 10  층정보       200833 non-null  object 
 11  호정보       0 non-null       float64
 12  경도        330827 non-null  float64
 13  위도        330827 non-null  float64
dtypes: float64(3), int64(1), object(10)
memory usage: 35.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521721 entries, 0 to 521720
Data columns (total 14 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   --

In [30]:
# 결측치가 있는 변수는 type에 맞게 fillna 해줍니다.
Seoul_22_1Q["지점명"] = Seoul_22_1Q["지점명"].fillna("0")
Seoul_22_1Q["표준산업분류명"] = Seoul_22_1Q["표준산업분류명"].fillna("0")
Seoul_22_1Q["동정보"] = Seoul_22_1Q["동정보"].fillna("0")
Seoul_22_1Q["층정보"] = Seoul_22_1Q["층정보"].fillna("0")
Seoul_22_1Q["호정보"] = Seoul_22_1Q["호정보"].fillna(0)

Gyeonggi_22_1Q["지점명"] = Seoul_22_1Q["지점명"].fillna("0")
Gyeonggi_22_1Q["표준산업분류명"] = Seoul_22_1Q["표준산업분류명"].fillna("0")
Gyeonggi_22_1Q["동정보"] = Seoul_22_1Q["동정보"].fillna("0")
Gyeonggi_22_1Q["층정보"] = Seoul_22_1Q["층정보"].fillna("0")
Gyeonggi_22_1Q["호정보"] = Seoul_22_1Q["호정보"].fillna(0)

In [32]:
# Seoul 상가 위치 시각화 해봅니다.
'''
plt.figure(figsize=(17,12))
ax = sns.scatterplot(data=Seoul_22_1Q, x='경도',y='위도', hue="시군구명")
plt.step(ax.get_legend().get_texts())
plt.show()
'''
# VS Code에서는 그림 잘 나왔었는데 에러가 뜨네요.. 일단은 넘어갈게요

'\nplt.figure(figsize=(17,12))\nax = sns.scatterplot(data=Seoul_22_1Q, x=\'경도\',y=\'위도\', hue="시군구명")\nplt.step(ax.get_legend().get_texts())\nplt.show()\n'

In [37]:
# pivot table을 만들어서 보겠습니다.
pivot_Seoul_22_1Q = pd.pivot_table(data=Seoul_22_1Q, index="시군구명", columns="상권업종대분류명", values="상가업소번호", aggfunc=['count'], margins=True, margins_name='합계')
pivot_Seoul_22_1Q

count                                                        
상권업종대분류명 관광/여가/오락    부동산  생활서비스     소매    숙박   스포츠      음식  학문/교육      합계
시군구명                                                                     
강남구           677   1884   6499   7894   148   672   11115   2313   31202
강동구           363    755   2932   3414    63   233    4502   1239   13501
강북구           270    258   1616   2467    91   107    3205    506    8520
강서구           449    855   3406   4001    93   299    5968   1219   16290
관악구           387    641   2446   3050    79   204    4338    907   12052
광진구           401    680   2560   3832    67   203    4302   1013   13058
구로구           296    539   2506   3805   103   177    4102    943   12471
금천구           198    446   1902   3055    71   122    2885    520    9199
노원구           302    425   2120   3233    27   201    3784   1371   11463
도봉구           184    291   1510   2340    18   136    2479    716    7674
동대문구          255    530   2103   4491   112   148    3775    720   12134
동작구           246    477   1851   2468    35   137    3109    902    9225
마포구           293    695   3052   3260   118   201    7173    936   15728
서대문구          195    368   1604   2410    79   132    3649    654    9091
서초구           465   1166   3837   5752    70   410    5877   1535   19112
성동구           235    515   2003   2854    53   176    3634    632   10102
성북구           242    469   2240   3376    78   153    3918   1027   11503
송파구           567   1019   3773   5777    63   390    6869   1578   20036
양천구           303    475   2382   3186    22   189    3473   1503   11533
영등포구          335    601   2563   4397   146   217    5133    711   14103
용산구           213    603   1655   3324   109   107    4075    405   10491
은평구           269    566   2387   3115    52   155    4065   1013   11622
종로구           222    389   1374   6270   203    93    4755    493   13799
중구            197    374   2240   8114   271   129    4556    328   16209
중랑구           338    464   2302   3194    87   128    3452    744   10709
합계           7902  15485  62863  99079  2258  5119  114193  23928  330827

서울 지역별 분석

1) 강남구 31202개로 압도적 1위입니다.

다음으로 송파구 20036개, 서초구 19112개, 강서구 16290개 이지만 강남 3구로 묶입니다. 다 같은 동네라는 말이죠...

업종을 불문, 상가 집중도가 엄청난 곳인 것 같습니다...

2) 중구가 16209개로 많은 편인데... 중구가 서울역부터 명동, 을지로까지 포함하는 동네라 많은 듯 합니다.

중구가 숙박이 가장 많은 구라는 점이 재밌는데, 중국인 관광객을 위한 숙소가 많은 것으로 알고 있습니다.

3) 홍대는 어떤가 해서 봤더니 마포구 15728개로 역시 많은 편이네요.

--------------

서울 업종별 분석

1) 330827개 중 "음식점"이 114193개로 약 1/3 차지합니다.

2) "소매"가 99079개인데, 옷가게, 서점, 마트 등 음식점이 아닌 가게를 모으다보니 많은 것 같습니다.

3) "생활서비스"가 62863개인데 대부분 "미용/건강"입니다. 가게 이름 확인해보니 헤어샵이 대부분이네요.

헬스장은 스포츠로 분류되어서 "생활서비스"에 포함되지 않는데도 이 정도면... 헤어샵이 참 많은 것 같습니다.



In [40]:
# 서울 내 업종별 분석에서 1위한 "음식"만 떼어내서 중분류명으로 pivot table 보겠습니다.
pivot_Seoul_food_22_1Q = pd.pivot_table(data=Seoul_22_1Q[(Seoul_22_1Q["상권업종대분류명"]=="음식")], index="시군구명", columns="상권업종중분류명", values="상가업소번호", aggfunc=['count'], margins=True, margins_name='합계')
pivot_Seoul_food_22_1Q

count                                                          \
상권업종중분류명  기타음식업  닭/오리요리 별식/퓨전요리     부페      분식      양식     유흥주점 음식배달서비스   
시군구명                                                                      
강남구        44.0   337.0   212.0   47.0   740.0  1149.0    851.0    11.0   
강동구         6.0   270.0   101.0   25.0   415.0   141.0    477.0     4.0   
강북구         3.0   218.0    68.0    6.0   304.0    84.0    391.0     3.0   
강서구        28.0   326.0   123.0   18.0   488.0   324.0    583.0     1.0   
관악구        12.0   253.0    99.0   13.0   411.0   250.0    549.0     3.0   
광진구        13.0   228.0    89.0   12.0   395.0   195.0    446.0     1.0   
구로구        14.0   278.0   103.0   17.0   374.0    98.0    559.0     3.0   
금천구        12.0   170.0    83.0   14.0   261.0    82.0    323.0     1.0   
노원구        13.0   304.0    87.0    8.0   389.0   126.0    352.0     7.0   
도봉구         7.0   180.0    50.0   11.0   258.0    79.0    233.0     1.0   
동대문구        7.0   211.0    89.0   15.0   354.0   112.0    378.0     5.0   
동작구        13.0   186.0    72.0   13.0   313.0   111.0    315.0     1.0   
마포구        19.0   235.0   151.0   19.0   410.0   421.0    725.0     4.0   
서대문구       23.0   181.0    81.0    9.0   353.0   227.0    337.0     2.0   
서초구        22.0   218.0   142.0   19.0   506.0   380.0    440.0     5.0   
성동구        27.0   152.0    85.0    3.0   261.0   235.0    284.0     4.0   
성북구        17.0   243.0    84.0    7.0   362.0   181.0    327.0     2.0   
송파구        21.0   310.0   143.0   27.0   575.0   285.0    541.0     8.0   
양천구        11.0   208.0    75.0   16.0   344.0    99.0    356.0     2.0   
영등포구       26.0   223.0   122.0   37.0   398.0   236.0    557.0     3.0   
용산구        12.0   127.0   101.0    7.0   262.0   524.0    407.0     5.0   
은평구        11.0   256.0    88.0   14.0   379.0   127.0    456.0     NaN   
종로구        19.0   156.0    94.0   13.0   416.0   422.0    338.0     3.0   
중구         21.0   168.0   107.0   39.0   413.0   247.0    427.0     5.0   
중랑구         9.0   242.0    82.0   10.0   336.0    82.0    404.0     1.0   
합계        410.0  5680.0  2531.0  419.0  9717.0  6217.0  11056.0    85.0   

                                                                    
상권업종중분류명  일식/수산물 제과제빵떡케익      중식   커피점/카페   패스트푸드       한식      합계  
시군구명                                                                
강남구        950.0   448.0   300.0   2086.0   501.0   3439.0   11115  
강동구        352.0   221.0   113.0    721.0   243.0   1413.0    4502  
강북구        189.0   115.0    61.0    343.0   150.0   1270.0    3205  
강서구        446.0   254.0   179.0   1055.0   295.0   1848.0    5968  
관악구        320.0   146.0   114.0    660.0   216.0   1292.0    4338  
광진구        319.0   181.0   151.0    758.0   226.0   1288.0    4302  
구로구        237.0   149.0   231.0    560.0   200.0   1279.0    4102  
금천구        172.0   103.0   120.0    455.0   127.0    962.0    2885  
노원구        236.0   204.0   118.0    563.0   270.0   1107.0    3784  
도봉구        176.0   114.0    68.0    372.0   133.0    797.0    2479  
동대문구       241.0   137.0   136.0    508.0   179.0   1403.0    3775  
동작구        233.0   164.0    86.0    536.0   172.0    894.0    3109  
마포구        434.0   305.0   165.0   1438.0   252.0   2595.0    7173  
서대문구       199.0   176.0    92.0    678.0   212.0   1079.0    3649  
서초구        432.0   252.0   166.0   1157.0   255.0   1883.0    5877  
성동구        236.0   141.0   108.0    636.0   161.0   1301.0    3634  
성북구        225.0   165.0    94.0    678.0   239.0   1294.0    3918  
송파구        560.0   306.0   181.0   1121.0   377.0   2414.0    6869  
양천구        244.0   184.0    94.0    579.0   228.0   1033.0    3473  
영등포구       366.0   181.0   242.0    881.0   226.0   1635.0    5133  
용산구        232.0   152.0    93.0    691.0   171.0   1291.0    4075  
은평구        264.0   209.0   114.0    573.0   185.0   1389.0    4065  
종로구        262.0   148.0   110.0   1009.0   149.0   1616.0    4755  
중구         289.0   170.0   125.0    879.

114193개 중 한식이 37194개로 약 1/3차지합니다.

카페가 19422개, 술집이 11056개이고요.

그냥 비중이 이렇다~ 정도 파악하면 될 것 같습니다.

+닭/오리 따로 떼어낸거 웃긴데 꽤 많아서 놀랐습니다. 치킨집이 정말 많네요

In [39]:
# 경기도로 넘어가보겠습니다.
pivot_Gyeonggi_22_1Q = pd.pivot_table(data=Gyeonggi_22_1Q, index="시군구명", columns="상권업종대분류명", values="상가업소번호", aggfunc=['count'], margins=True, margins_name='합계', fill_value = 0)
pivot_Gyeonggi_22_1Q

count                                                         
상권업종대분류명 관광/여가/오락    부동산  생활서비스      소매    숙박   스포츠      음식  학문/교육      합계
시군구명                                                                      
가평군            61    147    471    1255  1024    79    1997    140    5174
고양시            22     22     62      82     4    14      59     20     285
고양시 덕양구       325    701   2582    4053    63   219    4567   1155   13665
고양시 일산동구      300    540   2248    3807    31   204    3768    853   11751
고양시 일산서구      193    431   1698    3203    52   150    2814   1155    9696
과천시            25    103    302     723     3    25     574    156    1911
광명시           213    481   1674    2990    33   122    2612   1014    9139
광주시           448    864   2912    5874   112   229    5056   1146   16641
구리시           171    376   1448    2837    57   104    2563    674    8230
군포시           184    299   1262    1852    33   137    2295    673    6735
김포시           405   1044   3351    5907    84   295    6540   1751   19377
남양주시          456   1289   4402    8310   187   401    8362   2283   25690
동두천시           79    168    835    1510    64    46    1624    282    4608
부천시           904   1390   5388    9732   137   392    7392   2431   27766
성남시 분당구       442   1155   3273    6043    52   380    5803   2241   19389
성남시 수정구       246    486   1768    2633   126   103    3026    528    8916
성남시 중원구       269    449   1776    2867   240    92    2678    566    8937
수원시             0      3      0       2     0     0       0      0       5
수원시 권선구       384    558   2790    3982    59   191    3659    998   12621
수원시 영통구       253    693   2153    3132    19   203    3835   1424   11712
수원시 장안구       257    431   1879    2571    47   133    2692    924    8934
수원시 팔달구       313    540   2068    4110   239   135    3630    652   11687
시흥시           561    982   3240    5946   113   276    6327   1444   18889
안산시             0      2      2       8     0     0       5      0      17
안산시 단원구       497    604   2733    4964   222   206    4749    980   14955
안산시 상록구       399    487   2552    3926    69   159    3636    924   12152
안성시           193    324   1300    2561    83   109    3399    543    8512
안양시 동안구       347    777   2435    4074    62   224    3667   1347   12933
안양시 만안구       279    425   1767    3327    99   109    2775    703    9484
양주시           155    458   1678    3048   150   134    3862    657   10142
양평군           127    422    879    1698   396    55    2563    291    6431
여주시           120    248    840    2021   124    71    2623    255    6302
연천군            35     96    346     838    98    15     990     87    2505
오산시           237    328   1681    2663   105   126    3153    702    8995
용인시             4      8     22      29     0     1      33      8     105
용인시 기흥구       271    761   2496    3975    52   319    4159   1635   13668
용인시 수지구       203    653   1682    2845    23   221    2768   1303    9698
용인시 처인구       265    548   1955    3799   159   143    4303    767   11939
의왕시            83    214    627    1113    16    76    1281    355    3765
의정부시          397    645   3171    5339   114   216    5328   1508   16718
이천시           287    529   2008    4709   152   151    4741    948   13525
파주시           438    900   3400    6235   205   225    7434   1432   20269
평택시           601   1413   4263    7683   207   267    8498   1691   24623
포천시           214    350   1391    3851   310   107    4091    313   10627
하남시           197    587   2040    3668    43   181    3752    742   11210
화성시           693   1858   5037    9580   269   515   10830   2606   31388
합계          12553  25789  91887  165375  5737  7560  170513  42307  521721

경기도 지역별 분석

0) 고양시, 수원시 이런 곳은 전처리 과정에서 문제가 있었나봅니다. 저렇게 적지 않을텐데요... 

다시 확인해보겠습니다.

1) 화성시(31388개)와 바로 옆 평택시(24263개)에 "음식" & "소매"가 많이 있다고 합니다.

그리고 화성시와 인접한 지역은 안산(약 25000개), 시흥(18889개)이 있는데 두 곳 다 상권이 큰 것 같습니다.

2) 다음으로 부천시가 27766개로 많은데, 부천도 상권이 꽤 큽니다.

단순히 인천 가는 길이라 그런가 해서 찾아봤는데요,

길주로라는 길이 있는데, 2km 안에 백화점 2개, 대형마트 4개가 있는 밀집 지역이라고 하네요.

3) 남양주도 25690개로 많습니다.

2000년대 중반 이후 개발이 빠르게 진행되어 인구가 늘어나면서 상권도 발달한 것 같습니다.

-------
제가 경기도를 안다녀봐서 잘 모르는 것 같습니다

혹시 인사이트 있으면 나누어주세요~